# 0. Depedencies

In [ ]:
import numpy as np
from scipy.optimize import minimize
import plotly.graph_objects as go 
import plotly.express as px #for colors
from datetime import datetime
import datetime
import pandas as pd
import logging
from entsoe import EntsoePandasClient


In [ ]:
light_green = '#228A83'
dark_green = '#2E5651'
blue = '#002060'
brown = '#7A5F3F'

# 1. Data

## Production and consumption data

Watt-hour (Wh) is a measure of energy, not the average instantaneous power. It represents the total amount of energy consumed or produced over a period of time.

In [ ]:
# Convert the 5-minute watt values to watt-hours!!
# df['consumption'] = df['consumption'] * (5 / 60)
# df['production'] = df['production'] * (5 / 60)
# Now you can sum this up (could also have taken average I think) diviser par 20

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data
df = pd.read_csv('data/robbe.csv')

# Ensure the 'time' column is datetime and create a 'date' column
df['time'] = pd.to_datetime(df['time'])
df['date'] = df['time'].dt.date

# Group by date and sum the production and consumption values for each day
df_daily = df.groupby('date').agg({'production': 'sum', 'consumption': 'sum'}).reset_index()

# Plot total production and consumption per day
plt.figure(figsize=(12, 6))
plt.plot(df_daily['date'], df_daily['production'], label='Total Production')
plt.plot(df_daily['date'], df_daily['consumption'], label='Total Consumption')
plt.xlabel('Date')
plt.ylabel('Total Power (Wh)')
plt.title('Total Production and Consumption per Day')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.show()


In [ ]:
# date_to_tune = '2023-03-09'
date_to_tune = '2023-05-21'

#make it datetime
date_to_tune_datetime = pd.to_datetime(date_to_tune)

In [ ]:
df = pd.read_csv('data/robbe.csv')
# in Watt!!!
df['date'] = pd.to_datetime(df['time']).dt.date
df['date'] = pd.to_datetime(df['date'])
df = df[df['date'] == date_to_tune] #change here date, less production
df = df.drop(columns=['date'])
df = df.reset_index(drop=True)

df['time'] = pd.to_datetime(df['time']).dt.hour
df = df.groupby(['time']).mean().reset_index() #mean so it is fine
df = df.set_index('time')
#df is still in Watt

production = df['production'].values # in W
consumption = df['consumption'].values  # in W

production_kW = production/1000
consumption_kW = consumption/1000
#print total prod and cons
total_prod_kW = np.sum(production_kW)
total_cons_kW = np.sum(consumption_kW)
print('Total production: ', total_prod_kW)
print('Total consumption: ', total_cons_kW)
#similar prod anc consumption, explain not always like this

## Price data

In [ ]:
date_to_tune_datetime

In [ ]:
# Replace with your actual API key
entsoe_api_key = 'eaccd9d9-2a8e-42b0-9257-2aa47096934e'

class DayAheadPrices:
    def __init__(self):
        self.client = EntsoePandasClient(api_key=entsoe_api_key)

    def fetch_and_print_day_ahead_prices(self, country_code='BE'):
        """Fetches and prints the day ahead prices for Belgium."""
        # Set the date range for today
        timezone = 'Europe/Brussels'

        desired_date = date_to_tune_datetime

        # Set date_start to the beginning of the desired date
        date_start = pd.Timestamp(datetime.datetime(desired_date.year, desired_date.month, desired_date.day, 0, 0, 0), tz=timezone)

        # Set date_end to the end of the desired date
        date_end = pd.Timestamp(datetime.datetime(desired_date.year, desired_date.month, desired_date.day, 23, 59, 59), tz=timezone)

        # date_start = pd.Timestamp(datetime.date.today() - datetime.timedelta(days=2), tz=timezone)
        # date_end = pd.Timestamp(datetime.date.today() - datetime.timedelta(days=1), tz=timezone)

        logging.info(f'Fetching day ahead prices from ENTSO-E for {country_code}: {date_start} to {date_end}')
        
        # Fetch day ahead prices
        try:
            day_ahead_prices = self.client.query_day_ahead_prices(country_code=country_code, start=date_start, end=date_end) #cannot pass it like this resolution=['15T'] and also no quantities
            print("Day Ahead Prices (€/MWh):")
            print(day_ahead_prices)
        except Exception as e:
            print(f"An error occurred while fetching data: {e}")

        return day_ahead_prices

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    day_ahead_prices = DayAheadPrices()
    prices = day_ahead_prices.fetch_and_print_day_ahead_prices()


In [ ]:
prices = prices.values

In [ ]:
prices_MWh = prices
prices_kWh = prices_MWh / 1000
prices_Wh = prices_kWh / 1000

In [ ]:
# prices_kWh = np.load('data/prices_robbe.npy') 
prices = prices_kWh #for the moment
consumption = consumption_kW
production = production_kW

In [ ]:
# for mismatching
original_prices = prices.copy()
original_production = production.copy()
original_consumption = consumption.copy()

# 2. MPC

## 2.1 Helpers

In [ ]:
# Utility functions defined outside the class
def total(price, volume):
    return price * volume

#for mismatching
def generate_profiles(time_segments, probability): #add pprobab as param
        # np.random.seed(42)

        if np.random.rand() > probability:  # 90% of the time, add this as a parameter later
                profile = np.full(time_segments, 1 / time_segments)  # Uniformly distributed
        else:
            random_values = np.random.uniform(0, 1, time_segments)
            random_values /= random_values.sum()  # Ensure sum is 1
            profile = np.round(random_values, decimals=1)  # Rounded to one decimal

        return profile

In [ ]:
def mean_flatten_list(list_of_lists):
    flattened_list = [item if isinstance(sublist, list) else sublist for sublist in list_of_lists for item in (sublist if isinstance(sublist, list) else [sublist])]
    #make numpy array
    flattened_list = np.array(flattened_list)
    #mean of list
    mean_list = np.mean(flattened_list)
    return round(mean_list,2)

## 2.2 Class EMS problem

In [ ]:
class OptimizationProblem: #energy setup

    def __init__(self, soc_initial, electricity_prices, pv_production, consumption, N, horizon, max_capacity, max_charge_rate, grid_price_ratio, time_segments, method, options, num_guesses, probs_mismatch, accuracy_forecast, mismatching_occurence):
        
        self.soc_initial = soc_initial
        self.H = horizon
        self.N = N #not bigger than one here

        self.electricity_prices = electricity_prices
        self.pv_production = pv_production #first in object creation
        self.consumption = consumption

        self.max_capacity = max_capacity
        self.max_charge_rate = max_charge_rate
        self.max_discharge_rate = -max_charge_rate
        self.efficiency_battery = 1

        self.time_segments = time_segments
        self.probs_mismatch = probs_mismatch
        self.grid_price_ratio = grid_price_ratio
        self.accuracy_forecast = accuracy_forecast
        self.mismatching_occurence = mismatching_occurence

        self.method = method
        self.options = options
        self.num_guesses = num_guesses

        #-------------------

        self.true_charging_actions = None  # To store the "true" charging actions
        self.soc_list = None  # To store the state of charge over time
        self.grid_list = None  # To store the net grid interaction over time

        #for looping over best guesses
        self.best_actions = None
        self.best_soc = None
        self.best_grid = None
        self.best_fake_actions = None

        self.evaluations = [] #for guesses convergence

    def get_bounds(self):
        # bounds based on max_charge_rate and max_discharge_rate
        return [(self.max_discharge_rate * self.N, self.max_charge_rate * self.N) for _ in range(self.H)]
   

    def initial_guesses(self): #smart guesses!!
        current_soc = self.soc_initial
        if self.num_guesses == 1:
            # One guess: neutral middle-ground, rounded to two decimals.
            return [np.round(np.zeros(self.H), 2)]
        else:
            guesses = []
            for _ in range(self.num_guesses):
                guess = np.zeros(self.H)
                for t in range(self.H):
                    # Randomly decide between discrete choices and a continuous range selection.
                    if np.random.rand() < 0.25:
                        # 20% chance to select any value in the range.
                        action = np.random.uniform(self.max_discharge_rate * self.N, self.max_charge_rate * self.N)
                    else:
                        # 80% chance to favor extremes and zero.
                        if current_soc == 0:
                            # Can only charge_rate.
                            action = np.random.choice([0, self.max_charge_rate * self.N], p=[0.2, 0.8])
                        elif current_soc == self.max_capacity:
                            # Can only discharge_rate.
                            action = np.random.choice([0, self.max_discharge_rate * self.N], p=[0.2, 0.8])
                        else:
                            # Can charge_rate or discharge_rate; favor extremes and zero.
                            action = np.random.choice([0, self.max_charge_rate * self.N, self.max_discharge_rate * self.N], p=[0.4, 0.3, 0.3])

                    # Round the action to two decimals.
                    action = np.round(action, 2)
                    
                    # Apply action to the current state of charge_rate, ensuring it remains within bounds.
                    current_soc = np.clip(current_soc + action, 0, self.max_capacity)
                    guess[t] = action
                    
                    # Optionally, enforce more realistic sequences (e.g., discharge_rate after charging).
                    if t > 0 and guess[t-1] > 0 and action > 0:
                        # If previously charging, introduce a higher chance to switch to discharging or neutral.
                        if np.random.rand() < 0.3:
                            guess[t] = np.random.choice([0, self.max_discharge_rate * self.N], p=[0.5, 0.5])
                            guess[t] = np.round(guess[t], 2)

                    elif t > 0 and guess[t-1] < 0 and action < 0:
                        # If previously discharging, introduce a higher chance to switch to charging or neutral.
                        if np.random.rand() < 0.3:
                            guess[t] = np.random.choice([0, self.max_charge_rate * self.N], p=[0.5, 0.5])
                            guess[t] = np.round(guess[t], 2)

                    # probability is set to 0.3 in both cases
                    
                    current_soc = np.clip(current_soc + guess[t] - action, 0, self.max_capacity)
                
                guesses.append(guess)
            
            print('the length of the guesses', len(guesses))
            print('these are the guesses', guesses)
            
            return guesses
        
    def cost_function(self, proposed_charging_actions):
        soc = self.soc_initial
        charging_cost = 0
        soc_list = [soc]  # Initialize SOC list with initial SOC
        charging_action_list = []
        net_grid = []

        #constraint 2   

        for i in range(self.H): #at each timestep

            #a) action -------------------------------------
            if proposed_charging_actions[i] > 0:  
                charging_action = min(proposed_charging_actions[i], (1 - soc) * self.max_capacity) 
                #proposed action is already bounded via self.N
            elif proposed_charging_actions[i] < 0: 
                charging_action = max(proposed_charging_actions[i], -soc * self.max_capacity) #maximum of two negative values. Quesrion: can I send signal to battery to discharge more?
            else:  
                charging_action = 0
            
            #b) state of charge -------------------------------------
            if charging_action > 0:
                real_action = charging_action * self.efficiency_battery 
                soc += real_action / self.max_capacity
            elif charging_action < 0:  # When you ask a battery to discharge a certain power, taking into account its efficiency, it needs to discharge more than requested to ensure the desired output is met due to efficiency losses
                real_action = charging_action / self.efficiency_battery
                soc += real_action / self.max_capacity
            else:
                real_action = 0
            
            #c) net grid -------------------------------------
            # constraint 1
                
            # grid = self.consumption[i] - self.pv_production[i] + (real_action * self.H)/1000 #to put in kWh
            grid = self.consumption[i] - self.pv_production[i] + real_action #to put in kWh


            if grid > 0:
                charging_cost += grid * self.electricity_prices[i]
            elif grid < 0:
                charging_cost += grid * self.grid_price_ratio * self.electricity_prices[i]
            else:
                pass

            if self.N < 1:
                charging_cost = charging_cost * self.N #because price is per hour eur/kWh

            # append to lists
            soc_list.append(round(soc,2))
            charging_action_list.append(round(real_action, 2))
            net_grid.append(round(grid,2))

        #only useful when efficiency...
        self.true_charging_actions = charging_action_list 
        self.soc_list = soc_list

        #interesting
        self.grid_list = net_grid
        self.evaluations.append((charging_cost))

        print('cost', charging_cost)

        return charging_cost
        # Return the total charging cost as the optimization objective (only one variable allowed)
    

    def apply_mismatch_and_update_soc(self, prod_amount, cons_amount, proposed_action, soc_init): #not needed to be in class I think, but better for params
        #for one action each time 
            
        if self.probs_mismatch == 0: #no mismatching
            net_grid = cons_amount - prod_amount + proposed_action
            soc_init = soc_init + proposed_action / self.max_capacity
            return soc_init, net_grid, proposed_action

        else:
            soc = soc_init #not same as self.soc_initial
            net_grid_segment = 0
            from_grid = 0
            to_grid = 0
            total_grid = []
            actions_segment_list = []

            # Generate mismatch profiles
            if self.mismatching_occurence == 0: 
                prod_mismatch_profile = generate_profiles(self.time_segments, self.probs_mismatch)
                cons_mismatch_profile = generate_profiles(self.time_segments, self.probs_mismatch)
            elif self.mismatching_occurence == 1:
                prod_mismatch_profile = generate_profiles(self.time_segments, self.probs_mismatch)
                cons_mismatch_profile = np.full(self.time_segments, 0)
            else: #self.mismatching_occurence == 2
                prod_mismatch_profile = np.full(self.time_segments, 0)
                cons_mismatch_profile = generate_profiles(self.time_segments, self.probs_mismatch)

            soc_list = []
            # Iterate through each time segment
            for i in range(self.time_segments):
                # print(f'this is the {i}th iteration')
                segment_prod = prod_amount * prod_mismatch_profile[i]
                segment_cons = cons_amount * cons_mismatch_profile[i]

                if proposed_action > 0:  #proposed action already bounded
                    action_for_segment = min(proposed_action * prod_mismatch_profile[i], self.max_charge_rate * self.N/self.time_segments) #rmber self.N = 1 if N >= 1
                    soc += action_for_segment / self.max_capacity
                    soc_list.append(soc)

                else:  
                    action_for_segment = max(proposed_action * cons_mismatch_profile[i], self.max_discharge_rate * self.N/self.time_segments) #same comments, max of two negative numbers
                    soc += action_for_segment / self.max_capacity
                    soc_list.append(soc)
                
                # Update net grid exchange
                net_grid_segment = segment_cons - segment_prod + action_for_segment

                if net_grid_segment > 0:
                    from_grid += net_grid_segment
                else:
                    to_grid += net_grid_segment

                actions_segment_list.append(action_for_segment)
        
                # soc = max(0, min(soc, 1)) * 100 #not needed normally

                #prints
                # print(f'new soc at end segment {round(soc * 100, 2)} % , action in segment: {round(action_for_segment,2)}, segment cons: {round(segment_cons,2)}, segment prod: {round(segment_prod,2)}, net grid: {round(net_grid_segment,2)}')
            
            total_grid = [from_grid, to_grid]

            return soc_list, total_grid, actions_segment_list
    

    #-------------------------- # not a lot different from cost function

    def basic_ruler(self):
        soc = self.soc_initial
        charging_cost = 0
        soc_list = [soc]  # does not impact
        charging_action_list = []
        net_grid = []
        buying_cost = []
        selling_cost = []
        mean_price = np.mean(self.electricity_prices)

        #constraint 2
        for i in range(len(self.consumption)): #at each timestep, nothing to do with horizon bcs kijkt niet op voorhand, mismatching is per action donc blc

            #a) action lined to price
            # -------------------------------------
            if self.electricity_prices[i] < mean_price:  # here self.N because not bounded

                # charging_action = min(self.max_charge_rate * self.N, (1 - soc) * self.max_capacity)
                charging_action = min(self.max_charge_rate, (1 - soc) * self.max_capacity) #without self.N because biased...

            
            elif self.electricity_prices[i] >= mean_price:

                # charging_action = max(self.max_discharge_rate * self.N, -soc * self.max_capacity)
                charging_action = max(self.max_discharge_rate, -soc * self.max_capacity)

            else:  
                charging_action = 0
            
            #b) state of charge
            # -------------------------------------
            if charging_action > 0:
                real_action = charging_action * self.efficiency_battery 
                soc += real_action / self.max_capacity

            elif charging_action < 0:  # When you ask a battery to discharge a certain power, taking into account its efficiency, it needs to discharge more than requested to ensure the desired output is met due to efficiency losses
                real_action = charging_action / self.efficiency_battery
                soc += real_action / self.max_capacity
            else:
                real_action = 0
            
            #c) net grid
            # -------------------------------------
            # constraint 1
            grid = self.consumption[i] - self.pv_production[i] + real_action

            #chargig_cost is aggregate, later use ut to calculate buying and selling
            if grid > 0:
                charging_cost += grid * self.electricity_prices[i]
                buying_cost.append(charging_cost)
            elif grid < 0:
                charging_cost += grid * self.grid_price_ratio * self.electricity_prices[i]
                selling_cost.append(charging_cost)
            else:
                pass

            # append to lists, if interested
            soc_list.append(round(soc,2))
            charging_action_list.append(round(real_action, 2))
            net_grid.append(round(grid,2))
        
        if self.N < 1:
                charging_cost = charging_cost * self.N


        sum_buying = np.sum(buying_cost)
        sum_selling = np.sum(selling_cost)
        print(f'this is the total buying {sum_buying}')
        print(f'this is the total selling {sum_selling}')

        # #print sum of list of net grid
        net_grid = np.array(net_grid)
        print('basic ruler net grid sum', net_grid.sum())
        print(f'soc_list: {soc_list}')
        print(f'charging_action_list: {charging_action_list}')

        return net_grid, charging_cost

## 2.3 Optimisation algo at each timestep - minimise

In [ ]:
def optimize_battery_operation(problem: OptimizationProblem, soc):
    
    initial_guesses = problem.initial_guesses()
    # print('this are the initial guesses', initial_guesses)

    bounds = problem.get_bounds() #will need to be changed later
    best_result = None

    list_nfev = [] #colled in other cell
    all_cost_evolutions = [] #will only be used if timestep ==0

    def callback(xk):
        current_cost = problem.cost_function(xk)
        cost_evolution.append(current_cost)

    i= 0
    for guess in initial_guesses:

        problem.soc_initial = soc

        cost_evolution = []

        result = minimize(
        problem.cost_function, 
        guess,  
        method= problem.method, 
        bounds=bounds,
        options= problem.options,
        callback=callback
    )
        print('result', result, 'of guess', guess, 'number', i)
        
        i = i + 1

        # print(f"Best result: {round(result.fun,2)} Euro")
        # print(f'Number of nfev: {result.nfev}')
        list_nfev.append(result.nfev)

        # print(f'Cost evolution at guess {i}, {cost_evolution}')
        all_cost_evolutions.append(cost_evolution)

        if best_result is None or result.fun < best_result.fun:
            best_result = result
            problem.best_actions = problem.true_charging_actions

            #below not really needed
            problem.best_fake_actions = result.x
            problem.best_actions = problem.true_charging_actions
            problem.best_soc = problem.soc_list
            problem.best_grid = problem.grid_list
    
    # print(f"Best result: {round(best_result.fun,2)} Euro")
    
    return list_nfev, all_cost_evolutions

## 2.4 Cost computation

In [ ]:
def cost_calculator_no_mismatching(net_grid, electricity_prices, ratio_sell_back, timestep_duration):
    # input: net_grid, ratio_sell_back, prices
    total_cost = 0
    buying_cost = 0
    selling_cost = 0

    if len(net_grid) != len(electricity_prices):
        repeat_factor = len(electricity_prices) // len(net_grid)
        electricity_prices = np.mean(electricity_prices.reshape(-1, repeat_factor), axis=1)
        # print(f'Length of net_grid and electricity_prices are set the same {electricity_prices}')
        print(f'Length of net_grid and electricity_prices are set the same')


    #teken price influences
    for i in range(len(net_grid)):
        # print(f'timestep------------: {i}') #printed and is correct, prices afgestemd
        if net_grid[i] > 0:
            # print('BUYING')
            # print(f'net grid: {net_grid[i]}')
            # print(f'prices {electricity_prices[i]}')
            buying_cost += net_grid[i] * electricity_prices[i]
            # print(f'new buying cost: {buying_cost}')
        elif net_grid[i] < 0:
            # print('SELLING')
            # print(f'net grid: {net_grid[i]}')
            # print(f'prices corrected {electricity_prices[i]* ratio_sell_back}')
            # print(f'prices corrected {electricity_prices[i]* 0.25}')
            selling_cost += net_grid[i] * ratio_sell_back * electricity_prices[i] #negative
            # print(f'new selling cost: {selling_cost}')
        else:
            pass

    if timestep_duration < 1: #see data aggregation
        buying_cost = buying_cost * timestep_duration
        selling_cost = selling_cost * timestep_duration

    total_cost = buying_cost + selling_cost

    return total_cost, buying_cost, selling_cost

In [ ]:
def cost_calculator_with_mismatching(net_grid, electricity_prices, ratio_sell_back, timestep_duration):
    # input: net_grid, ratio_sell_back, prices
    total_cost = 0
    buying_cost = 0
    selling_cost = 0
    
    if len(net_grid) != len(electricity_prices):
        repeat_factor = len(electricity_prices) // len(net_grid)
        electricity_prices = np.mean(electricity_prices.reshape(-1, repeat_factor), axis=1)
        print(f'Length of net_grid and electricity_prices are set the same {electricity_prices}')
    
    print('len net grid', len(net_grid))
    print('this is net grid before', net_grid)
    buy, sell = zip(*net_grid)
    # print(f'buy: {buy}, sell: {sell}')
    
    buying_cost = np.sum(buy * electricity_prices)
    selling_cost = np.sum(sell * electricity_prices) * ratio_sell_back

    if timestep_duration < 1: #see data aggregation
        buying_cost = buying_cost * timestep_duration
        selling_cost = selling_cost * timestep_duration

    print('buy', buying_cost)
    print('sell', selling_cost)

    total_cost = buying_cost + selling_cost #changed it, selling neg so automatically

    return total_cost, buying_cost, selling_cost

## 2.5 Forecast variability

In [ ]:
def stochastically_deviate(array, deviation_range_percentage):
    
    # For each element, calculate a deviation percentage within the allowed range
    deviations = np.random.uniform(-deviation_range_percentage, deviation_range_percentage, size=array.shape)
    
    # Apply the calculated deviations to the original array elements
    deviated_array = array + (array * deviations)
    
    return deviated_array

## 2.6 Simple heuristic 

In [ ]:
def simple_heuristic(prices, cons, prod, duration_timestep):
    mean_price = np.mean(prices)
    total_cons = np.sum(cons)
    total_prod = np.sum(prod)
    net_grid = total_cons - total_prod
    total_net_grid = np.sum(net_grid)
    cost = total_net_grid * mean_price
    if duration_timestep < 1:
        cost = cost * duration_timestep
    return round(cost,2)

# 3. Parameters


In [ ]:
#setup
battery_capacity = 10.2
max_charge = 3.5
max_discharge = -max_charge

sell_back_ratio = 0.3
forecast_certainty = 1

mismatching_occurence = 0 #both
timesteps_for_mismatching = 6 #quarter... or maybe more if
horiz = 6 #12 hour maybe
duration_step_in_hour = 1 #averaged for example

efficiency_battery = 1
efficiency_inverter = 1 
soc_start = 0 #initial level
# options = {'maxiter': 100, 'ftol': 0.001, 'disp': True, 'verbose': 2} #to be sure
method = 'SLSQP'

#to tweak
options = {'maxiter': 800, 'ftol': 0.001} 
num_guesses = 50 #a lot
mismatching_prob = 0 #later 0.9
#make two variables for the last_one

# 4. First timesep and benchmarks

## 4.1 Full day only first timestep for graph 4

In [ ]:
convergence_one_day = OptimizationProblem(soc_start, prices, production, consumption, duration_step_in_hour, 24, battery_capacity, max_charge, sell_back_ratio, timesteps_for_mismatching, method, options, 30, mismatching_prob, forecast_certainty, mismatching_occurence)

#initialize variables
# optimal_controls = []
# soc_evolution = [soc]
# net_grid_evolution = []
# number_iterations_per_time_step = []

horizon_copy = convergence_one_day.H
length_copy = len(convergence_one_day.electricity_prices)
soc = soc_start #not clean but blc


# timestep = 0
# for t in range(length_copy):
#     if timestep == 0:

list_nfev_guesses, iterations = optimize_battery_operation(convergence_one_day, convergence_one_day.soc_initial) #added here timestep but no needed

all_days_function_evals = list_nfev_guesses
all_days_iterations_evolution = iterations

print('these are the function evaluations', all_days_function_evals)
print('these are the iterations', all_days_iterations_evolution)

# number_iterations_per_time_step.append(list_nfev_guesses)

# new_soc, new_grid, new_action = convergence_one_day.apply_mismatch_and_update_soc(convergence_one_day.pv_production[0], convergence_one_day.consumption[0], convergence_one_day.best_actions[0], soc) #depends on mismatch prob

# optimal_controls.append(new_action)
# net_grid_evolution.append(new_grid)
# soc_evolution.append(new_soc)

# convergence_one_day.soc_initial = new_soc

    # else: 
    #     pass
    # timestep = timestep + 1

#no cost calc but return cost

## 4.2 Benchmarks

In [ ]:
#print
mean_price_Wh = round(np.mean(prices_Wh),2)
net_grid = consumption - production # in Watt
net_grid_no_pv = consumption # in Watt
sum_net_grid = round(np.sum(net_grid), 2)
print('mean price Wh', mean_price_Wh)
print('net grid', net_grid)
print('net grid no pv', net_grid_no_pv)
print('sum net grid', sum_net_grid)

#more tangible in kWh!


### A. No PV, No Battery

In [ ]:
# total_simple_heuristic = simple_heuristic(prices, consumption, production, duration_step_in_hour)
# #print and round
# print('total simple heuristic', round(total_simple_heuristic,2))

In [ ]:
total_no_battery_no_pv, buy_no_battery_no_pv, sell_no_battery_no_pv = cost_calculator_no_mismatching(net_grid_no_pv, prices, sell_back_ratio, duration_step_in_hour)
#print
print('total no battery no pv', round(total_no_battery_no_pv,2))
print('buy no battery no pv', round(buy_no_battery_no_pv,2))
print('sell no battery no pv', round(sell_no_battery_no_pv,2))

### B. No battery

In [ ]:
net_grid

In [ ]:
total_pv_no_battery, buy_pv_no_battery, sell_pv_no_battery = cost_calculator_no_mismatching(net_grid, prices, sell_back_ratio, duration_step_in_hour)
#print 
print('total pv no battery', round(total_pv_no_battery,2))
print('buying pv no battery', round(buy_pv_no_battery,2))
print('selling pv no battery', round(sell_pv_no_battery,2))

#### C. Basic rule based on average price

In [ ]:
base_ruler = OptimizationProblem(soc_start, prices, production, consumption, duration_step_in_hour, horiz, battery_capacity, max_charge, sell_back_ratio, timesteps_for_mismatching, method, options, num_guesses, mismatching_prob, forecast_certainty, mismatching_occurence)
net, cost = base_ruler.basic_ruler() #changed this

In [ ]:
total_basic_ruler, buy_basic_ruler, sell_basic_ruler = cost_calculator_no_mismatching(net, prices, sell_back_ratio, duration_step_in_hour)
print('total cost basic ruler', round(total_basic_ruler,2))
print('buying cost basic ruler', round(buy_basic_ruler,2))
print('selling cost basic ruler', round(sell_basic_ruler,2))

#cost should be the same as total
#could make a graph from it


D. Overview

In [ ]:
# print('total simple heuristic', round(total_simple_heuristic,2))
#add the other
print('-----------------------')
print('total cost no battery', round(total_pv_no_battery,2))
print('buying cost no battery', round(buy_pv_no_battery,2))
print('selling cost no battery', round(sell_pv_no_battery,2))
print('-----------------------')
print('total cost basic ruler', round(total_basic_ruler,2))
print('buying cost basic ruler', round(buy_basic_ruler,2))
print('selling cost basic ruler', round(sell_basic_ruler,2))
#addd alter MPC

In [ ]:
# # genAI, avoid to load it/save it
# data = {
#     'time': pd.date_range(start='2023-02-26 00:00:00', end='2023-02-26 23:00:00', freq='H'),
#     'prices': [137.69, 126.35, 120.00, 109.85, 105.93, 106.37, 107.04, 107.04, 123.12, 123.85,
#                97.13, 92.67, 84.78, 55.13, 55.24, 84.90, 98.92, 129.90, 153.70, 156.91, 
#                153.64, 141.64, 138.29, 132.98]
# }
# df = pd.DataFrame(data)

# # Convert the prices column to a numpy array
# prices_MWh = df['prices'].values
# prices_kWh = prices_MWh / 1000
# prices_Wh = prices_kWh / 1000

# # Display the numpy array
# print(prices_kWh)

# 5. NO MISMATCHING

## 5.1 Code

Here the 'rolling forward' component

In [ ]:
rolling_problem = OptimizationProblem(soc, prices, production, consumption, duration_step_in_hour, horiz, battery_capacity, max_charge, sell_back_ratio, timesteps_for_mismatching, method, options, num_guesses, mismatching_prob, forecast_certainty, mismatching_occurence)

#initialize variables
optimal_controls = []
soc_evolution = [soc] # I do not know if need to add this...?
net_grid_evolution = []
number_iterations_per_time_step = []

horizon_copy = rolling_problem.H
length_copy = len(rolling_problem.electricity_prices)

timestep = 0
for t in range(length_copy):

    #initial 3 variables
    rolling_problem.electricity_prices = prices[t:t+horizon_copy]
    rolling_problem.pv_production = production[t:t+horizon_copy]
    rolling_problem.consumption = consumption[t:t+horizon_copy]

    if len(rolling_problem.electricity_prices) != horizon_copy: #at the end, assumption here
        rolling_problem.electricity_prices = np.append(rolling_problem.electricity_prices, np.zeros(horizon_copy - len(rolling_problem.electricity_prices)))
        rolling_problem.pv_production = np.append(rolling_problem.pv_production, np.zeros(horizon_copy - len(rolling_problem.pv_production)))
        rolling_problem.consumption = np.append(rolling_problem.consumption, np.zeros(horizon_copy - len(rolling_problem.consumption)))

    if rolling_problem.accuracy_forecast != 1:
        deviation = 1 - rolling_problem.accuracy_forecast
        rolling_problem.pv_production = stochastically_deviate(rolling_problem.pv_production, deviation)
        rolling_problem.consumption = stochastically_deviate(rolling_problem.consumption, deviation)
    
    print('another timestep --------------', timestep)
    list_nfev_guesses, iterations = optimize_battery_operation(rolling_problem, soc) #no need timestep  

    number_iterations_per_time_step.append(list_nfev_guesses) #for all, will need fro graph 4
    
    new_soc, new_grid, new_action = rolling_problem.apply_mismatch_and_update_soc(rolling_problem.pv_production[0], rolling_problem.consumption[0], rolling_problem.best_actions[0], soc) #depends on mismatch prob

    optimal_controls.append(new_action)
    net_grid_evolution.append(new_grid) 
    soc_evolution.append(new_soc)
    
    soc = new_soc

    timestep = timestep + 1 #just for printing

#----------------- cost calculation
if mismatching_prob == 0:
    MPC_total_no_mismatching, MPC_buying_no_mismatching, MPC_selling_no_mismatching = cost_calculator_no_mismatching(net_grid_evolution, prices, sell_back_ratio, duration_step_in_hour) #could also take from class
    avg_soc = np.mean(soc_evolution)
else:
    print('not possible')
    # MPC_total, MPC_buying, MPC_selling = cost_calculator_with_mismatching(net_grid_evolution, prices, sell_back_ratio, duration_step_in_hour)
    # avg_soc = mean_flatten_list(soc_evolution)

In [ ]:
#cost, put at the end for overview
print("MPC_total:", MPC_total_no_mismatching) 
print("MPC_buying:", MPC_buying_no_mismatching)
print("MPC_selling:", MPC_selling_no_mismatching)

## 5.2 Visualisation

### 5.2.1 Preprocessing

For preprocessing add copy to last elem

In [ ]:
def add_elem(array):
    if len(array) == 24:
        last_elem = array[-1]
        array = list(array)  # Convert NumPy array to list
        array.append(last_elem)
        array = np.array(array)  # Convert list back to NumPy array
    return array

In [ ]:
prices_for_visual = prices_kWh #before adding an element

In [ ]:
prices_for_visual = add_elem(prices_for_visual)
production = add_elem(production)
consumption = add_elem(consumption)
#prod and cons in kWh
production_kW = add_elem(production_kW)
consumption_kW = add_elem(consumption_kW)
net_grid_evolution = add_elem(net_grid_evolution)

### Graph 1: two graphs for production, consumption, net_grid, price

Change datum graph 1

In [ ]:
net = production - consumption
net

In [ ]:
# Create a figure
fig = go.Figure()

# left y-axis
fig.add_trace(go.Scatter(x=list(range(len(production_kW))),y=production_kW,mode='lines',name='production',line_shape='hv',line=dict(color=light_green, width=3)))
fig.add_trace(go.Scatter(x=list(range(len(consumption_kW))), y=consumption_kW, mode='lines', name='consumption',line_shape='hv', line=dict(color=blue, width=3)))

#right y-axis
fig.add_trace(go.Scatter(x=list(range(len(prices_for_visual))), y=prices_for_visual, mode='lines', name='Prices (right axis)', line_shape='hv', yaxis='y2', line=dict(color=dark_green, width=3)))


# Update the layout
fig.update_layout(title='Consumption, production Robbe  and DAP 21/05/2023',
                xaxis_title='Timesteps',
                yaxis=dict(
                title='Energy (kWh)'),
                yaxis2=dict(
                title='Price (Euro/kWh)',
                overlaying='y',
                side='right'),
                legend_title='Legend',
                legend=dict(
                orientation='h',  # horizontal orientation
                y=- 0.2,  # Position legend at the top (y < 1)
                x=0.1),
                font=dict(color=dark_green),
                title_x=0.5
                )
fig.show()

In [ ]:
# Create a figure
fig = go.Figure()

# left y-axis
fig.add_trace(go.Scatter(x=list(range(len(production_kW))),y=production_kW,mode='lines',name='production',line_shape='hv',line=dict(color=light_green, width=3)))
fig.add_trace(go.Scatter(x=list(range(len(consumption_kW))), y=consumption_kW, mode='lines', name='consumption',line_shape='hv', line=dict(color=blue, width=3)))

# Update the layout
fig.update_layout(title='Consumption and production Robbe 21/05/2023',
                xaxis_title='Timesteps',
                yaxis=dict(
                title='Energy (kWh)'), #so de facctor kWh but to check
                font=dict(color=dark_green),
                title_x=0.5,
                legend=dict(
                orientation='h',  # horizontal orientation
                y=- 0.1,  # Position legend at the top (y < 1)
                x=0.1))  # Title font color)
fig.show()

#change title
#legend above, units

In [ ]:
# Create a figure
fig = go.Figure()

# left y-axis
fig.add_trace(go.Scatter(x=list(range(len(net_grid_evolution))), y=net_grid_evolution, mode='lines', name='Net grid', line_shape='hv', line=dict(color=light_green, width=3)))
#check if elec sold

#right y-axis
fig.add_trace(go.Scatter(x=list(range(len(prices_for_visual))), y=prices_for_visual, mode='lines', name='Prices (right axis)', line_shape='hv', yaxis='y2', line=dict(color=blue, width=3)))

# Update the layout
fig.update_layout(title='No mismatching: day-ahead electricity prices and net grid instantaneous power over a daily period',
                xaxis_title='Timesteps',
                yaxis=dict(
                title='Instantaneous power (W)'),
                yaxis2=dict(
                title='Price (Euro/kWh)',
                overlaying='y',
                side='right'),
                legend_title='Legend',
                legend=dict(
                orientation='h',  # horizontal orientation
                y=- 0.2,  # Position legend at the top (y < 1)
                x=0.1),
                font=dict(color=dark_green),
                title_x=0.5
                )

# Show the figure
fig.show()

In [ ]:
net_grid_evolution_no_mismatching_copy = net_grid_evolution

### Graph 2: Actions & SOC battery

In [ ]:
optimal_controls = add_elem(optimal_controls) #tobe sure
soc_evolution = [x * 100 for x in soc_evolution]
fig = go.Figure()

'''NO MISMATCHING'''
# left y-axis
fig.add_trace(go.Scatter(x=list(range(len(optimal_controls))), y=optimal_controls, mode='lines', name='Battery actions without mismatching', line_shape='hv', line=dict(color=light_green, width=3)))
#right y-axis
fig.add_trace(go.Scatter(x=list(range(len(soc_evolution))), y=soc_evolution, mode='lines', name='SoC without mismatching', yaxis='y2', line=dict(color=blue, width=3))) #dark_green

'''MISMATCHING''' #outcommented
# fig.add_trace(go.Scatter(x=list(range(len(soc_evolution_with_mismatching))), y=soc_evolution_with_mismatching, mode='lines', name='SoC with mismatching (', yaxis='y2'))

# Create the x and y values for the continuous line
# x_values_continuous = []
# y_values_continuous = []

# for index, data_list in enumerate(optimal_controls_with_mismatching):
#     x_values_continuous.extend([index + i/(timesteps_for_mismatching -1) for i in range(len(data_list))])
#     y_values_continuous.extend(data_list)
# fig.add_trace(go.Scatter(x=x_values_continuous, y=y_values_continuous, mode='lines+markers', name='Battery actions with mismatching'))

# Update the layout
fig.update_layout(title='No mismatching: control actions and net state of charge over a daily period',
                xaxis_title='Timesteps',
                yaxis=dict(
                title='Instantaneous power (kW)'),
                yaxis2=dict(
                title='State of charge battery (%)',
                overlaying='y',
                side='right'),
                yaxis_title='Instantaneous power (W)',
                xaxis=dict(dtick=1),
                legend_title='Legend',
                legend=dict(
                orientation='h',  # horizontal orientation
                y=- 0.2,  # Position legend at the top (y < 1)
                x=0),    # Center the legend horizontally )
                font=dict(color=dark_green),
                title_x=0.5 
                )

# Show the figure
fig.show()

#### Add all together to see axis instanteneous power for perspective

In [ ]:
# Create a figure
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(len(production))),y=production,mode='lines',name='production',line_shape='hv',line=dict(color=light_green, width=2, dash='dash')))
fig.add_trace(go.Scatter(x=list(range(len(consumption))), y=consumption, mode='lines', name='consumption',line_shape='hv', line=dict(color=dark_green, width=2, dash='dash')))
fig.add_trace(go.Scatter(x=list(range(len(net_grid_evolution))), y=net_grid_evolution, mode='lines', name='Net grid', line_shape='hv', line=dict(color=brown, width=3)))
fig.add_trace(go.Scatter(x=list(range(len(prices_for_visual))), y=prices_for_visual, mode='lines', name='Prices (right axis)', line_shape='hv', yaxis='y2', line=dict(color=blue, width=3)))

# Update the layout
fig.update_layout(title='No mismatching: production, consumption, control actions, and net SoC over a daily period',
                xaxis_title='Timesteps',
                yaxis=dict(
                title='Instantaneous power (kW)'),
                yaxis2=dict(
                title='Price (Euro/kWh)',
                overlaying='y',
                side='right'),
                yaxis_title='Instantaneous power (W)',
                xaxis=dict(dtick=1),
                legend_title='Legend',
                legend=dict(
                orientation='h',  # horizontal orientation
                y=- 0.2,  # Position legend at the top (y < 1)
                x=0),    # Center the legend horizontally )
                font=dict(color=dark_green),
                title_x=0.5 
                )

fig.show()

## Graph 3: convergence

BLC for the moment

In [ ]:
list_lengths = []
for elem in all_days_iterations_evolution:
    length = len(elem)
    list_lengths.append(length)

In [ ]:
all_days_lengths_iterations = [len(sublist) for sublist in all_days_iterations_evolution]

In [ ]:
#25 times
all_days_with_nfev = [25 * x for x in all_days_lengths_iterations] #changes...

In [ ]:
#flatten all_days_iterations_evolution
def flatten_list_of_lists(lst, copy_number):
    flat_list = []
    for sublist in lst:
        for item in sublist:
            for _ in range(copy_number):  
                flat_list.append(item)
    return flat_list

flat_iterations = flatten_list_of_lists(all_days_iterations_evolution, 25)

In [ ]:
def split_into_sublists(flat_list, sublist_lengths):
    sublists = {}
    start = 0
    for i, length in enumerate(sublist_lengths, start=1):
        end = start + length
        # print(end)
        sublists[f'guess_{i}'] = flat_list[start:end]
        start = end
    return sublists #dictionary

original_sublists = split_into_sublists(flat_iterations,all_days_with_nfev) #two ouput from optimization, note list_nfev_guesses could be also in class

Only first 30 guesses

In [ ]:
# Initialize the figure outside the loop
fig = go.Figure()

# Assuming original_sublists and N are defined
for i in range(1, 30+1 ):
    fig.add_trace(go.Scatter(
        x=list(range(len(original_sublists[f'guess_{i}']))), 
        y=original_sublists[f'guess_{i}'], 
        mode='lines', 
        name=f'Guess {i}'
    ))

# Update layout once after all traces have been added
fig.update_layout(
    title='Visualisation of evolution initial guesses - objective function evaluations (no mismatching)',
    xaxis_title='Iterations',
    yaxis_title='Cost obtained by proposed actions (Euro)',
    font=dict(color=dark_green),
    title_x=0.5 
)
#tomar legend but quit ...
fig.show()


In [ ]:
total_basic_ruler = -total_basic_ruler
sell_basic_ruler = -buy_basic_ruler
buy_basic_ruler = -sell_basic_ruler

# 6. MISMATCHING

identical

## 6.1 param and running

In [ ]:
mismatching_prob = 0.9 #here, only that changes
prices = original_prices #strictly not needed
production = original_production 
consumption = original_consumption
#because has been modified to be sure

In [ ]:
rolling_problem = OptimizationProblem(soc, prices, production, consumption, duration_step_in_hour, horiz, battery_capacity, max_charge, sell_back_ratio, timesteps_for_mismatching, method, options, num_guesses, mismatching_prob, forecast_certainty, mismatching_occurence)

#initialize variables
optimal_controls = []
# soc_evolution = [[0,0,0,0]] #beug here
soc_evolution = [] #beug here
net_grid_evolution = []
number_iterations_per_time_step = []

horizon_copy = rolling_problem.H
length_copy = len(rolling_problem.electricity_prices)
soc = soc_start #not correct should work with class mais blc

timestep = 0
for t in range(length_copy):

    #initial 3 variables
    rolling_problem.electricity_prices = prices[t:t+horizon_copy]
    rolling_problem.pv_production = production[t:t+horizon_copy]
    rolling_problem.consumption = consumption[t:t+horizon_copy]

    if len(rolling_problem.electricity_prices) != horizon_copy: #at the end, assumption here
        rolling_problem.electricity_prices = np.append(rolling_problem.electricity_prices, np.zeros(horizon_copy - len(rolling_problem.electricity_prices)))
        rolling_problem.pv_production = np.append(rolling_problem.pv_production, np.zeros(horizon_copy - len(rolling_problem.pv_production)))
        rolling_problem.consumption = np.append(rolling_problem.consumption, np.zeros(horizon_copy - len(rolling_problem.consumption)))

    if rolling_problem.accuracy_forecast != 1:
        deviation = 1 - rolling_problem.accuracy_forecast
        rolling_problem.pv_production = stochastically_deviate(rolling_problem.pv_production, deviation)
        rolling_problem.consumption = stochastically_deviate(rolling_problem.consumption, deviation)
    
    print('another timestep --------------', timestep)
    list_nfev_guesses, iterations = optimize_battery_operation(rolling_problem, soc) #no need timestep  

    number_iterations_per_time_step.append(list_nfev_guesses) #for all, will need fro graph 4
    

    #here add during which timesteps possible... or based on production
    '''to complete here but problem different data structure'''

    # allowed_times = [9,10,11,12,13,14,15,16,17,18]
    # if timestep in allowed_times: #change it here, not in main function
    #     rolling_problem.probs_mismatch = 0.9 #variable in params does not play a role anymore
    #     new_soc, new_grid, new_action = rolling_problem.apply_mismatch_and_update_soc(rolling_problem.pv_production[0], rolling_problem.consumption[0], rolling_problem.best_actions[0], soc) #depends on mismatch prob
    # else:
    #     rolling_problem.probs_mismatch = 0
    
    new_soc, new_grid, new_action = rolling_problem.apply_mismatch_and_update_soc(rolling_problem.pv_production[0], rolling_problem.consumption[0], rolling_problem.best_actions[0], soc) #depends on mismatch prob
    
    # print('new_grid', new_grid)
    # print('new_action', new_action)
    optimal_controls.append(new_action)
    net_grid_evolution.append(new_grid) 
    soc_evolution.append(new_soc)
    
    # if mismatching!!
    soc = new_soc[-1]

    timestep = timestep + 1 #just for printing

#----------------- cost calculation
if mismatching_prob == 0:
    print('not possible')
    # MPC_total, MPC_buying, MPC_selling = cost_calculator_no_mismatching(net_grid_evolution, prices, sell_back_ratio, duration_step_in_hour) #could also take from class
    # avg_soc = np.mean(soc_evolution)
else:
    MPC_total_mismatching, MPC_buying_mismatching, MPC_selling_mismatching = cost_calculator_with_mismatching(net_grid_evolution, prices, sell_back_ratio, duration_step_in_hour)
    avg_soc = mean_flatten_list(soc_evolution)

# avg_nfev = mean_flatten_list(number_iterations_per_time_step)

In [ ]:
#prints MPC_total_mismatching
print("MPC_total:", MPC_total_mismatching) 
print("MPC_buying:", MPC_buying_mismatching)
print("MPC_selling:", MPC_selling_mismatching)

## 6.2 Graphs

### Graph 1: Prices and net grid

In [ ]:
#cost before
#net_grid_evolution = [from_grid, to_grid]
if isinstance(net_grid_evolution[0], list): #perform only once
    net_grid_evolution = [sum(inner_list) for inner_list in net_grid_evolution]

In [ ]:
#prices already fine
net_grid_evolution = add_elem(net_grid_evolution)
# prices = add_elem(prices)
#prices_for_visual already defined

# Create a figure
fig = go.Figure()

# left y-axis
fig.add_trace(go.Scatter(x=list(range(len(net_grid_evolution_no_mismatching_copy))), y=net_grid_evolution_no_mismatching_copy, mode='lines', name='Net grid (no mismatching)', line_shape='hv', line=dict(color=light_green, width=3)))
fig.add_trace(go.Scatter(x=list(range(len(net_grid_evolution)+1)), y=net_grid_evolution, mode='lines', name='Net grid (mismatching)', line_shape='hv', line=dict(color=dark_green, width=4, dash='dash')))

#right y-axis
fig.add_trace(go.Scatter(x=list(range(25)), y=prices_for_visual, mode='lines', name='Prices (right)', line_shape='hv', yaxis='y2', line=dict(color=blue, width=3)))

# Update the layout
fig.update_layout(title='Day-ahead electricity prices and net grid instantaneous power over a daily period',
                title_x=0.5,
                xaxis_title='Timesteps',
                yaxis=dict(
                title='Instantaneous power (W)'),
                yaxis2=dict(
                title='Price (Euro/kWh)',
                overlaying='y',
                side='right'),
                legend_title='Legend',
                legend=dict(
                orientation='h',  # horizontal orientation
                y=- 0.2,  # Position legend at the top (y < 1)
                x=0),
                font=dict(color=dark_green)    # Center the legend horizontally )
                )

# Show the figure
fig.show()

### Graph 2: Actions & SOC battery

In [ ]:
optimal_controls = add_elem(optimal_controls) #tobe sure

fig = go.Figure()


# '''NO MISMATCHING'''
# # left y-axis
# fig.add_trace(go.Scatter(x=list(range(len(optimal_controls))), y=optimal_controls, mode='lines', name='Battery actions without mismatching', line_shape='hv', line=dict(color=light_green)))
# #right y-axis
# fig.add_trace(go.Scatter(x=list(range(len(soc_evolution))), y=soc_evolution, mode='lines', name='SoC without mismatching', yaxis='y2', line=dict(color=blue))) #dark_green

'''MISMATCHING'''
#soc evol
# Create the x and y values for the continuous line
x_values_continuous = []
y_values_continuous = []

for index, data_list in enumerate(soc_evolution):
    x_values_continuous.extend([index + i/(timesteps_for_mismatching) for i in range(len(data_list))])
    data_list = [x * 100 for x in data_list]
    y_values_continuous.extend(data_list)
    
fig.add_trace(go.Scatter(x=x_values_continuous, y=y_values_continuous, mode='lines+markers', name='SoC with mismatching', yaxis='y2', line=dict(color=blue, width=3)))

#control actions
# Create the x and y values for the continuous line
x_values_continuous = []
y_values_continuous = []

for index, data_list in enumerate(optimal_controls):
    x_values_continuous.extend([index + i/(timesteps_for_mismatching) for i in range(len(data_list))])
    y_values_continuous.extend(data_list)
    
fig.add_trace(go.Scatter(x=x_values_continuous, y=y_values_continuous, mode='lines+markers', name='Battery actions with mismatching', line=dict(color=light_green, width=3)))

# Update the layout
fig.update_layout(title='Mismatching: control actions and battery state of charge over a daily period',
                title_x=0.5,
                xaxis_title='Timesteps',
                yaxis=dict(
                title='Instantaneous power (W)'),
                xaxis=dict(
                range=[0, 24]),
                yaxis2=dict(
                title='Battery level (%)',
                overlaying='y',
                side='right'),
                legend_title='Legend',
                legend=dict(
                orientation='h',  # horizontal orientation
                y=- 0.2,  # Position legend at the top (y < 1)
                x=0),
                font=dict(color=dark_green),    # Center the legend horizontally )
                )


# Show the figure
fig.show()

# 7. Final overview

In [ ]:
#mean of prices_kWh
mean_price_kWh = round(np.mean(prices_kWh),4)
print('mean price kWh', mean_price_kWh) #0.12 per kWh normally, OK

In [ ]:
#find where consumption is a bit higher than prod
print('Total production: ', round(total_prod_kW,2), 'kWh')
print('Total consumption: ', round(total_cons_kW,2), 'kWh')
#net grid
print('------')
net_grid = total_cons_kW - total_prod_kW
print('Simple heursitic normally: ', round(net_grid * mean_price_kWh,2)) #maybe get rid of it

In [ ]:
# print('total simple heuristic', round(total_simple_heuristic,4)) #not really useful, better no PV heuristic
print('total no battery no pv', round(total_no_battery_no_pv,4))
print('buy no battery no pv', round(buy_no_battery_no_pv,4))
print('sell no battery no pv', round(sell_no_battery_no_pv,4))
print('-----------------------')
print('total pv no battery', round(total_pv_no_battery,4))
print('buying pv no battery', round(buy_pv_no_battery,4))
print('selling pv no battery', round(sell_pv_no_battery,4))
print('-----------------------')
print('total cost basic ruler', round(total_basic_ruler,4))
print('buying cost basic ruler', round(buy_basic_ruler,4))
print('selling cost basic ruler', round(sell_basic_ruler,4))
print('-----------------------')
print("MPC_total mismatching:", round(MPC_total_no_mismatching,4)) 
print("MPC_buying mismatching:", round(MPC_buying_no_mismatching,4))
print("MPC_selling mismatching:", round(MPC_selling_no_mismatching,4))
print('-----------------------')
print("MPC_total no mismatching:", round(MPC_total_mismatching,4)) 
print("MPC_buying no mismatching:", round(MPC_buying_mismatching,4))
print("MPC_selling no mismatching:", round(MPC_selling_mismatching,4))